# Разработка простого LLM-приложения

В этом разделе показан пример разрбаотки LLM-приложения, которое переводит текст с английского языка на другой язык.
Итоговое приложение включает всего один вызов LLM плюс некоторую работу с промптами.
Это довольно простое приложение, но оно показывает, что множество функций можно реализовать только с помощью промптов и вызова LLM.

## Основные понятия

В разеделе расмотрены следующие основные понятия:

- использование [языковых моделей](/docs/concepts/#chat-models);

- использование [шаблонов промптов](/docs/concepts/#prompt-templates) и [парсеров вывода](/docs/concepts/#output-parsers);

- [объединение в цепочку](/docs/concepts/#langchain-expression-language) шаблона промптов + LLM + парсера вывода с помощью GigaChain

- Развертывание вашего приложения с [LangServe](/docs/concepts/#langserve).
<!--
- Отладка и трассировка вашего приложения с помощью [LangSmith](/docs/concepts/#langsmith)
-->

## Подготовка к разработке

### Jupyter-блокноты

Это руководство, как и большинство других в документации, использует [Jupyter-блокноты](https://jupyter.org/). Они отлично подходят для изучения работы с LLM-системами, так как предоставляют интерактивную среду для работы с руководствами и позволяют работать с непредвиденными ситуациями: недоступностью API, нетипичным выводом и другими.

Подробнее об установке jupyter — в [официальной документации](https://jupyter.org/install).

### Установка

Для установки GigaChain выполните команды:

```{=mdx}
import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>
```


Подробнее об установке — в разделе [Установка](https://developers.sber.ru/docs/ru/gigachain/get-started/installation).

<!--
### LangSmith

Многие приложения, которые вы создаете с помощью LangChain, будут содержать несколько шагов с многократными вызовами LLM.
По мере усложнения этих приложений становится важно иметь возможность инспектировать, что именно происходит внутри вашей цепочки или агента.
Лучший способ сделать это — с помощью [LangSmith](https://smith.langchain.com).

После регистрации по ссылке выше, убедитесь, что вы установили переменные среды для начала ведения журнала трассировок:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Или, если вы работаете в ноутбуке, вы можете установить их с помощью:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```
-->

## Разбор примера

В разделе приводится пример разработки приложения для перевода пользовательского ввода с одного языка на другой.

## Использование языковых моделей

В первую очередь нужно подключить языковую модель.
GigaChain поддерживает различные языковые модели, которые могут заменять друг друга.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

In [1]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

Сначала попробуйте использовать модель напрямую.
`ChatModel` — это экземпляры Runnable-интерфейса GigaChain, что означает, что для работы они они предоставляют стандартный интерфейс.
Для простого вызова модели, вы можете передать список сообщений в метод `.invoke`.

In [16]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fc5d7c88-9615-48ab-a3c7-425232b562c5-0')

<!--
Если мы включили LangSmith, мы можем увидеть, что этот запуск зарегистрирован в LangSmith, и просмотреть [трассировку LangSmith](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r).
-->

## Парсеры вывода OutputParsers

Ответ от модели — это экземпляр класса `AIMessage`.
Такое сообщение содержит строковый ответ вместе с дополнительной информацией об ответе.
Часто для работы нужен только строковый ответ.
Для его получения вы можете использовать парсер вывода.

Импортируйте простой парсер вывода.

In [17]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

Парсер можно использовать отдельно от других компонентов приложения.
Например, вы можете сохранить результат вызова языковой модели и затем передать его в парсер.

In [18]:
result = model.invoke(messages)

In [19]:
parser.invoke(result)

'Ciao!'

Тем не менее, гораздо чаще парсер вывода объединяется в цепочку с моделью.
В таком случае он будет вызываться каждый раз при обращении к цепочке.
Полученная цепочка будет принимать на вход тип данных языковой модели (строку или список сообщений) и возвращать тип данных парсера вывода (строка).

Для создания цепочки, используйте оператор `|`.
GigaChain использует этот оператор для объединения двух элементов вместе.

In [20]:
chain = model | parser

In [21]:
chain.invoke(messages)

'Ciao!'

<!--
Если мы теперь посмотрим на LangSmith, мы увидим, что цепочка состоит из двух шагов: сначала вызывается языковая модель, затем результат передается парсеру вывода. Мы можем увидеть [трассировку LangSmith](https://smith.langchain.com/public/f1bdf656-2739-42f7-ac7f-0f1dd712322f/r).
-->

## Шаблоны промптов

На данный момент в примере список сообщений передается напрямую в языковую модель.
Как правило список сообщений формируется из комбинации ввода пользователя и логики приложения.
Эта логика обычно принимает необработанный ввод и преобразует его в список сообщений, готовых для передачи в языковую модель.
Типичные преобразования включают добавление системного сообщения или изменение шаблона с учетом ввода пользователя.

Шаблоны промптов (PromptTemplates) — это конструкции GigaChain, которые принимают необработанный ввод пользователя и возвращают данные (промпт), готовые для передачи в языковую модель.

Создайте шаблон промпта, который будет принимать две пользовательские переменные:

- `language` — язык, на который нужно перевести текст;
- `text` — текст для перевода.

Для этого импортируйте `ChatPromptTemplate`:

In [23]:
from langchain_core.prompts import ChatPromptTemplate

Создайте строку, которая будет оформлена как системное сообщение:

In [24]:
system_template = "Translate the following into {language}:"

Теперь вы можете создать шаблон промпта.
Он будет состоять из комбинации `system_template` и шаблона для ввода текста.

In [25]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

Входными данными для этого шаблона является словарь.
Вы можем поэкспериментировать с этим шаблоном, чтобы увидеть, что он делает сам по себе.

In [27]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

Шаблон возвращает объект `ChatPromptValue`, который состоит из двух сообщений.
Вы можете получить доступ к напрямую сообщениямм с помощью метода `.to_messages()`:

In [28]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

Теперь объедините все три компонента вместе: шаблон, модель и парсер вывода.

In [29]:
chain = prompt_template | model | parser

In [30]:
chain.invoke({"language": "italian", "text": "hi"})

'ciao'

<!--
Если мы посмотрим на трассировку LangSmith, мы увидим все три компонента в [трассировке LangSmith](https://smith.langchain.com/public/bc49bec0-6b13-4726-967f-dbd3448b786d/r).
-->

## Развертывание с LangServe

GigaServe помогает развертывать цепочки GigaChain в виде REST API.
Использовать GigaServe для работы с GigaChain необязательно.
Тем не менее ниже приводится пример, как вы можете развернуть приложение с помощью GigaServe.

Пример использует Python-файл, работа с которым вполняется с помощью командной строки.

Установка GigaServe:
```bash
pip install "langserve[all]"
```

### Сервер

Для создания сервера для приложения, создайте файл `serve.py`.
Файл будет содеражать логику для развертывания приложения.
Файл состоит из трех частей:

1. Определение цепочки, которую вы создали выше.
2. Приложение FastAPI.
3. Определение пути для доступа к цепочкие с помощью `langserve.add_routes`.

```python
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes

# 1. Создание шаблона промпта
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Создание модели
model = ChatOpenAI()

# 3. Создание парсера
parser = StrOutputParser()

# 4. Создание цепочки
chain = prompt_template | model | parser

# 4. Определение приложения
app = FastAPI(
  title="GigaChain Server",
  version="1.0",
  description="Простой сервер API, использующий Runnable-интерфейсы GigaChain.",
)

# 5. Добавление пути цепочки

add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)
```

Запустите файл:
```bash
python serve.py
```
Вы должны увидеть, что ваша цепочка доступна по адресу [http://localhost:8000](http://localhost:8000).

### Песочница

Приложения GigaServe предоставляют доступ к простому [пользовательскому интерфейсу](https://github.com/langchain-ai/langserve/blob/main/README.md#playground) для настройки и вызова приложения с потоковым выводом и отображением промежуточных шагов.
Вы можете попробовать его по адресу [http://localhost:8000/chain/playground/](http://localhost:8000/chain/playground/).
Введите такие же входные данные, как и раньше — `{"language": "italian", "text": "hi"}` — приложение должно ответить так же, как и раньше.

### Клиент

Для настройки клиентской части используйте `[langserve.RemoteRunnable](/docs/langserve/#client)`.
Так вы сможете взаимодействовать с доступной цепочкой так, как если бы она выполнялась на стороне клиента.

```python
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})
```

<CodeOutputBlock lang="python">

```python
    'Ciao'
```

</CodeOutputBlock>

Подробнее — в [документации GigaServe](/docs/langserve).

In [31]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

'Ciao'

To learn more about the many other features of LangServe [head here](/docs/langserve).

## Смотрите также

Чтобы глубже погрузиться в разработку приложений с помощью GigaChain, ознакомьтесь с разделами:

- [Обучающие материалы](/docs/tutorials);
- [Руководства](/docs/how_to);
- [Основные понятия](/docs/concepts).